In [1]:
%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

from utils import Connection

In [2]:
with Connection() as connection:
    age_groups_df = pd.read_sql('SELECT * FROM "TUKGRP2"."EX3_AGEGROUPS"', connection)

with Connection() as connection:
    avg_visits_per_age_df = pd.read_sql('SELECT * FROM "TUKGRP2"."EX3_AVG_VISITS_PER_AGE"', connection)

In [3]:
x = (age_groups_df['AGEGROUP'] * 10) - 5
y = age_groups_df['AVGVISITS']
X = x[:, np.newaxis]

x_plot = np.linspace(10, 95)
X_plot = x_plot[:, np.newaxis]

ages = avg_visits_per_age_df['AGE']
avg_visits_actual = avg_visits_per_age_df['AVGVISITS']

results = []

for degree in range(1, 11):
    interpolation_model = make_pipeline(PolynomialFeatures(degree), Ridge())
    interpolation_model.fit(X, y)
    y_plot = interpolation_model.predict(X_plot)
    avg_visits_prediction = interpolation_model.predict(ages[:, np.newaxis])
    mse = mean_squared_error(avg_visits_actual, avg_visits_prediction)
    print('d{}: MSE: {}'.format(degree, mse))
    results.append({ 'MSE': mse, 'DEGREE': degree,
                    'INTERPOLATION': ','.join(map(str, y_plot)),
                    'PREDICTEDVISITS': ','.join(map(str, avg_visits_prediction))})

d1: MSE: 1.638501838827966
d2: MSE: 1.6351121430512523
d3: MSE: 1.6364783853139684
d4: MSE: 1.6067706377581281
d5: MSE: 1.5786173292926735
d6: MSE: 1.538882557753805
d7: MSE: 1.4316931856274735
d8: MSE: 1.4450661389949404
d9: MSE: 1.7487473577482835
d10: MSE: 1.786723863272556


In [4]:
def export_results(results):
    query_template = """
    INSERT INTO "EX3_1_EXPORTS" ("DEGREE", "MSE", "INTERPOLATION", "PREDICTEDVISITS")
    VALUES ({}, {}, '{}', '{}')"""
    with Connection() as connection:
        cursor = connection.cursor().execute('TRUNCATE TABLE "EX3_1_EXPORTS"')
        for result in results:
            query = query_template.format(result['DEGREE'], result['MSE'],
                                          result['INTERPOLATION'], result['PREDICTEDVISITS'])
            cursor = connection.cursor().execute(query)
        connection.commit()
    
export_results(results)